# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 1 from hw3-t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [7]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [8]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [9]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [10]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    G = graph
    G.add_node('sink')
    for state in G.nodes():
        if state != 'CA' and state != 'sink':
            G.add_edge(state, 'sink')
            G.add_edge('sink', state)
            G.edge[state]['sink']['capacity'] = 1
            G.edge['sink'][state]['capacity'] = 0
    
    for (s1,s2) in G.edges():
        G.edge[s1][s2]['flow'] = 0
    
    def search(graph, s, t):
        for node in graph.nodes():
            graph.node[node]['discovered'] = 0
            graph.node[node]['parent'] = 'NIL'
        queue = [s]
        graph.node[s]['discovered'] = 1
        while len(queue) > 0:
            u = queue.pop(0)
            for v in graph.edge[u]:
                if graph.node[v]['discovered'] == 0 and graph.edge[u][v]['capacity'] != 0:
                    graph.node[v]['discovered'] = 1
                    graph.node[v]['parent'] = u
                    queue.append(v)
        return graph.node[t]['discovered']
    
    flow_max = 0
    while search(G, 'CA', 'sink') ==1:
        j = 'sink'
        while j != 'CA':
            i = G.node[j]['parent']
            G.edge[i][j]['flow'] += 1
            G.edge[i][j]['capacity'] -= 1
            G.edge[j][i]['capacity'] += 1
            j = G.node[j]['parent']
        flow_max += 1
        
    if flow_max != abs(G.node['CA']['demand']):
        raise nx.NetworkXUnfeasible("There Is No Flow Satisfying The Demands")
        
    for node in G.nodes():
        if node != 'CA' and node != 'sink':
            G.edge[node]['sink']['flow'] = 0
            
    for (s1, s2) in G.edges():
        del G[s1][s2]['capacity']
        
    return G

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [11]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    divrg = {}
    for state in G.nodes():
        divrg[state] = 0
    
    for (s1,s2) in flow.edges():
        divrg[s1] -= flow[s1][s2]['flow']
        divrg[s2] += flow[s1][s2]['flow']
    
    del divrg['sink']
    del G.node['sink']
    
    return divrg

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [12]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
